Case 1: OSM2GMNS bike-walk only

This notebook will show the process of generating a network from scratch using OSM2GMNS, generate the demand using GRID2DEMAND, and display some of the results from DTALite. Please note that the OSM file used here was downloaded using the OpenStreetMap website.

This process can be replicated by downloading this notebook along with the minimum input folder.

In [1]:
from __future__ import absolute_import
import osm2gmns as og
import subprocess
from grid2demand import GRID2DEMAND
import pandas as pd

osm2gmns, 0.7.3
grid2demand, version 0.3.8


This step assigns the instructions about various parameters such as OSM file name, mode types, POI (point of interest, which is necessary to generate the demand), etc., to a variable

In [2]:
net = og.getNetFromFile('asu.osm', 
                        network_types=('bike','walk'), 
                        POI=True,
                        POI_sampling_ratio=1,
                        strict_mode=True, 
                        min_nodes=10,
                        default_lanes=True, 
                        default_speed=True, 
                        default_capacity=True, 
                        start_node_id=0, 
                        start_link_id=0)

arguments used for network parsing:
  filename: asu.osm
  network_types: ('bike', 'walk')
  link_types: all
  POI: True
  POI_sampling_ratio: 1
  strict_mode: True
  offset: no
  min_nodes: 10
  combine: False
  bbox: None
  default_lanes: True
  default_speed: True
  default_capacity: True
  start_node_id: 0
  start_link_id: 0

Building Network from OSM file
  reading osm file
  parsing osm network
    generating nodes and links
    generating POIs
    removing sub networks with less than 10 nodes
  number of nodes: 2626, number of links: 4806, number of pois: 629


This step defines the default lanes, speed and capacity for links that do not contain such information

In [3]:
default_lanes_dict = {'motorway': 4, 'trunk': 3, 'primary': 3, 'secondary': 2, 'tertiary': 2,
                      'residential': 1, 'service': 1, 'cycleway':1, 'footway':1, 'track':1,
                      'unclassified': 1, 'connector': 2}
default_speed_dict = {'motorway': 120, 'trunk': 100, 'primary': 80, 'secondary': 60, 'tertiary': 40,
                      'residential': 30, 'service': 30, 'cycleway':5, 'footway':5, 'track':30,
                      'unclassified': 30, 'connector':120}
default_capacity_dict = {'motorway': 2300, 'trunk': 2200, 'primary': 1800, 'secondary': 1600, 'tertiary': 1200,
                      'residential': 1000, 'service': 800, 'cycleway':800, 'footway':800, 'track':800,
                      'unclassified': 800, 'connector':9999}

This step connects all the POI nodes to the nearest nodes using connector links

In [4]:
og.connectPOIWithNet(net)

This step generates the activities based on the POI information

In [5]:
og.generateNodeActivityInfo(net)

Generating Node Activity Information


This step generates the node, link and POI (if enabled) files in CSV format

In [6]:
og.outputNetToCSV(net)

Outputting Network Files


This step generates the demand files based on the POI file using GRID2DEMAND

Note: Please manually delete the entire column 'name' from poi.csv as this is known to cause issues when using pandas or GRID2DEMAND

In [7]:
if __name__ == "__main__":
    
    input_dir = ''
    gd = GRID2DEMAND(input_dir)

    # Step 1: Load node and poi data from input directory
    node_dict, poi_dict = gd.load_network.values()

    # Step 2: Generate zone dictionary from node dictionary by specifying number of x blocks and y blocks
    zone_dict = gd.net2zone(node_dict, num_x_blocks=10, num_y_blocks=10)
 
    # Step 3: synchronize geometry info between zone, node and poi
    updated_dict = gd.sync_geometry_between_zone_and_node_poi(zone_dict, node_dict, poi_dict)
    zone_dict_update, node_dict_update, poi_dict_update = updated_dict.values()

    # Step 4: Calculate zone-to-zone od distance matrix
    zone_od_distance_matrix = gd.calc_zone_od_distance_matrix(zone_dict_update)

    # Step 5: Generate poi trip rate for each poi
    poi_trip_rate = gd.gen_poi_trip_rate(poi_dict_update)

    # Step 6: Generate node production attraction for each node based on poi_trip_rate
    node_prod_attr = gd.gen_node_prod_attr(node_dict_update, poi_trip_rate)

    # Step 6.1: Calculate zone production and attraction based on node production and attraction
    zone_prod_attr = gd.calc_zone_prod_attr(node_prod_attr, zone_dict_update)

    # Step 7: Run gravity model to generate agent-based demand
    df_demand = gd.run_gravity_model(zone_prod_attr, zone_od_distance_matrix)

    # Step 8: generate agent-based demand
    df_agent = gd.gen_agent_based_demand(node_prod_attr, zone_prod_attr, df_demand=df_demand)

    # Step 9: Output demand, agent, zone, zone_od_dist_table, zone_od_dist_matrix files
    gd.save_demand
    gd.save_agent
    gd.save_zone
    gd.save_zone_od_dist_table
    gd.save_zone_od_dist_matrix

  : Input directory is not specified. Use current working directory C:/Users/aditr as input directory. Please make sure node.csv and poi.csv are in C:/Users/aditr.
  : Checking input directory...
  : input dir C:/Users/aditr, traverse files by type: csv
  : Input directory is valid.

  : Loading package settings...
  : Package settings loaded successfully.

INFO Begin to run function: read_network …
  : input dir C:/Users/aditr, traverse files by type: csv
INFO Begin to run function: read_node …
  : Parallel creating Nodes using Pool with 16 CPUs. Please wait...
  : Reading node.csv with specified columns: ['node_id', 'x_coord', 'y_coord', 'activity_type', 'is_boundary', 'poi_id']                 
    and chunksize 100000 for iterations...
  : Successfully loaded node.csv: 3255 Nodes loaded.
INFO Finished running function: read_node, total: 1s

INFO Begin to run function: read_poi …
  : Reading poi.csv with specified columns: ['poi_id', 'building', 'centroid', 'area', 'geometry']      

Run DTALite executable

In [8]:
command = 'DTALite'
try:
    subprocess.run(command, shell=True, check=True)
    print("DTALite executed successfully.")
except subprocess.CalledProcessError as e:
    print(f"Error executing DTALite: {e}")

DTALite executed successfully.


Display the total volume from link performance for both bike and walk modes

In [9]:
df = pd.read_csv('link_performance.csv')
x = df['vehicle_vol_walk'].sum()
x

8329.518

In [10]:
y = df['vehicle_vol_bike'].sum()
y

8755.812